In [1]:
import pandas as pd

forest_file_path = r"C:\Users\VAMSI\Desktop\project\_Cleaned_Forest_df_.csv"
grassland_file_path = r"C:\Users\VAMSI\Desktop\project\_Cleaned_GRASSLAND_df_.csv"

# Read the Excel file and covert to df
forest_df = pd.read_csv(forest_file_path)

grassland_df = pd.read_csv(grassland_file_path)

In [2]:
forest_df.info(), grassland_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8546 entries, 0 to 8545
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Admin_Unit_Code              8546 non-null   object 
 1   Site_Name                    8546 non-null   object 
 2   Plot_Name                    8546 non-null   object 
 3   Location_Type                8546 non-null   object 
 4   Year                         8546 non-null   int64  
 5   Date                         8546 non-null   object 
 6   Start_Time                   8546 non-null   object 
 7   End_Time                     8546 non-null   object 
 8   Observer                     8546 non-null   object 
 9   Visit                        8546 non-null   int64  
 10  Interval_Length              8546 non-null   object 
 11  ID_Method                    8546 non-null   object 
 12  Distance                     8546 non-null   object 
 13  Flyover_Observed  

(None, None)

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np

# Database configuration
config = {
    'host': 'mysql-3cfdc572-avamsi2k11-4e7b.h.aivencloud.com',
    'user': 'avnadmin',
    'password': 'AVNS_ozTCcwHYoNvj53twyRY',
    'port': 22799
}

def create_connection():
    """Create a database connection"""
    try:
        conn = mysql.connector.connect(**config)
        return conn
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

def create_tables(conn):
    """Create tables based on DataFrame structure"""
    
    try:
        cursor = conn.cursor()

        cursor.execute("create database if not exists project_guvi_bird_analysis")
        cursor.execute("USE project_guvi_bird_analysis")
        
        # Create forest_data table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS forest_data (
            Sno INT AUTO_INCREMENT PRIMARY KEY,
            Admin_Unit_Code CHAR(5),
            Site_Name VARCHAR(10),
            Plot_Name VARCHAR(15),
            Location_Type CHAR(15),
            Year INT,
            Date DATETIME,          
            Start_Time TIME,
            End_Time TIME,
            Observer CHAR(30),
            Visit INT,
            Interval_Length VARCHAR(20),
            ID_Method CHAR(30),
            Distance VARCHAR(10),
            Flyover_Observed BOOLEAN,
            Sex CHAR(20),
            Common_Name CHAR(100),
            Scientific_Name CHAR(100),
            AcceptedTSN INT,
            NPSTaxonCode INT,
            AOU_Code CHAR(10),
            PIF_Watchlist_Status BOOLEAN,
            Regional_Stewardship_Status BOOLEAN,
            Temperature FLOAT,
            Humidity FLOAT,
            Sky VARCHAR(100),
            Wind VARCHAR(100),
            Disturbance VARCHAR(100),
            Initial_Three_Min_Cnt BOOLEAN,
            SEASON CHAR(15),
            Month INT   
        )
        """)
        
        # Create grassland_data table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS grassland_data (
            Sno INT AUTO_INCREMENT PRIMARY KEY,
            Admin_Unit_Code CHAR(5),
            Plot_Name VARCHAR(15),
            Location_Type CHAR(15),
            Year INT,
            Date DATETIME,        
            Start_Time TIME,
            End_Time TIME,
            Observer CHAR(30),
            Visit INT,
            Interval_Length VARCHAR(20),
            ID_Method CHAR(30),
            Distance VARCHAR(10),
            Flyover_Observed BOOLEAN,
            Sex CHAR(20),
            Common_Name CHAR(100),
            Scientific_Name CHAR(100),
            AcceptedTSN INT,
            TaxonCode INT,
            AOU_Code CHAR(10),
            PIF_Watchlist_Status BOOLEAN,
            Regional_Stewardship_Status BOOLEAN,
            Temperature FLOAT,
            Humidity FLOAT,
            Sky VARCHAR(100),
            Wind VARCHAR(100),
            Disturbance VARCHAR(100),
            Previously_Obs BOOLEAN,          
            Initial_Three_Min_Cnt BOOLEAN,
            Month INT,
            SEASON CHAR(15)
                    
        )
        """)
        
        conn.commit()
        print("Tables created successfully")
        
    except Error as e:
        print(f"Error creating tables: {e}")
        conn.rollback()

def convert_numpy_types_to_python(df):
    """Convert NumPy data types to native Python types"""
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()
    
    for column in df_copy.columns:
       
        if pd.api.types.is_datetime64_any_dtype(df_copy[column]):
            df_copy[column] = df_copy[column].dt.to_pydatetime()
        
         # Convert int64/int32 to Python int
        elif pd.api.types.is_integer_dtype(df_copy[column]):
            df_copy[column] = df_copy[column].astype(int)
        
        # Convert float64 to Python float
        elif pd.api.types.is_float_dtype(df_copy[column]):
            df_copy[column] = df_copy[column].astype(float)
        
        # Convert boolean numpy values to Python bool
        elif pd.api.types.is_bool_dtype(df_copy[column]):
            df_copy[column] = df_copy[column].astype(bool)
            
    return df_copy

def insert_data_from_dataframe(conn, df, table_name):

    ####Insert data from DataFrame into specified table
    try:
        cursor = conn.cursor()
        
        # Convert column names to lowercase
        #df.columns = df.columns.str.lower()#
        
        # Convert NumPy data types to Python native types
        df = convert_numpy_types_to_python(df)
        
        # Create column list for SQL
        columns = ', '.join(df.columns)
        
        # Create placeholders for values
        placeholders = ', '.join(['%s'] * len(df.columns))
        
        # Convert DataFrame rows to list of tuples
        data = [tuple(row) for _, row in df.iterrows()]
        
        # SQL query
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        # Execute batch insert
        cursor.executemany(query, data)
        conn.commit()
        print(f"Inserted {cursor.rowcount} rows into {table_name}")
        
    except Error as e:
        print(f"Error inserting into {table_name}: {e}")
        conn.rollback()

def main():
    # Create connection
    conn = create_connection()
    if conn is None:
        return
    
    try:
        # Create tables
        create_tables(conn)
        
        # Insert data 
        if 'forest_df' in globals():
            insert_data_from_dataframe(conn, forest_df, 'forest_data')
        
        if 'grassland_df' in globals():
            insert_data_from_dataframe(conn, grassland_df, 'grassland_data')
            
    finally:
        if conn.is_connected():
            conn.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    main()


Tables created successfully
Inserted 8546 rows into forest_data
Inserted 6826 rows into grassland_data
MySQL connection is closed
